In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [22]:
csv0 = pd.read_csv("csv0.txt")
csv1 = pd.read_csv("csv1.txt")
csv2 = pd.read_csv("csv2.txt")
csv3 = pd.read_csv("csv3.txt")
csv4 = pd.read_csv("csv4.txt")
csv5 = pd.read_csv("csv5.txt")

/var/folders/c1/jbzkmqzx2b77zm2g8791t2wr0000gn/T/ipykernel_46597/3406777797.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  csv0 = pd.read_csv("csv0.txt")
/var/folders/c1/jbzkmqzx2b77zm2g8791t2wr0000gn/T/ipykernel_46597/3406777797.py:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  csv2 = pd.read_csv("csv2.txt")
/var/folders/c1/jbzkmqzx2b77zm2g8791t2wr0000gn/T/ipykernel_46597/3406777797.py:4: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  csv3 = pd.read_csv("csv3.txt")
/var/folders/c1/jbzkmqzx2b77zm2g8791t2wr0000gn/T/ipykernel_46597/3406777797.py:5: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  csv4 = pd.read_csv("csv4.txt")


In [23]:
df = pd.concat([csv0, csv1, csv2, csv3, csv4, csv5], ignore_index=True)

In [24]:
df

,game_id,player_id,team,won,agent,rating,average_combat_score,kills,deaths,assists,kills_deaths,kill_assist_trade_survive_ratio,average_damage_per_round,headshot_ratio,first_kills,first_deaths,side,tier,rounds_won
0,179438,Add3r,DZ,False,Killjoy,0.73,245,10,11,1,-1.0,75%,151.0,45%,1.0,0.0,t,21,2
1,179438,Add3r,DZ,False,Killjoy,1.15,353,4,3,0,1.0,100%,204.0,38%,0.0,0.0,ct,21,0
2,179438,Rawkus,DZ,False,Breach,0.90,180,7,10,7,-3.0,75%,119.0,42%,1.0,1.0,t,21,2
3,179438,Rawkus,DZ,False,Breach,0.28,35,0,3,1,-3.0,67%,35.0,0%,0.0,0.0,ct,21,0
4,179438,corey,DZ,False,Neon,0.67,140,6,10,2,-4.0,50%,104.0,38%,1.0,3.0,t,21,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
791175,16034,Retla,PRX,False,Reyna,0.78,251,10,11,1,-1.0,67%,121.0,9%,0.0,1.0,ct,21,5
791176,16034,lenne,PRX,False,Sova,0.58,104,4,9,3,-5.0,67%,68.0,6%,1.0,0.0,t,21,6
791177,16034,lenne,PRX,False,Sova,0.81,153,7,8,2,-1.0,33%,92.0,19%,0.0,3.0,ct,21,5
791178,16034,shiba,PRX,False,Brimstone,0.86,206,8,9,0,-1.0,42%,150.0,31%,3.0,1.0,t,21,6


In [25]:
df.dtypes

game_id                              int64
player_id                           object
team                                object
won                                   bool
agent                               object
rating                             float64
average_combat_score                object
kills                                int64
deaths                               int64
assists                              int64
kills_deaths                       float64
kill_assist_trade_survive_ratio     object
average_damage_per_round           float64
headshot_ratio                      object
first_kills                        float64
first_deaths                       float64
side                                object
tier                                 int64
rounds_won                           int64
dtype: object

### Preprocessing

In [26]:
from sklearn.model_selection import train_test_split

In [27]:
df = df.dropna()

In [28]:
X = df.drop(['won'], axis=1)
y = df.won

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=81)

### Training Models

In [30]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, FunctionTransformer, OneHotEncoder
from sklearn.impute import SimpleImputer


from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error

In [31]:
X_train.columns

Index(['game_id', 'player_id', 'team', 'agent', 'rating',
       'average_combat_score', 'kills', 'deaths', 'assists', 'kills_deaths',
       'kill_assist_trade_survive_ratio', 'average_damage_per_round',
       'headshot_ratio', 'first_kills', 'first_deaths', 'side', 'tier',
       'rounds_won'],
      dtype='object')

In [32]:
columns_to_drop = ["average_combat_score", "game_id", "player_id", "team", "rounds_won", "rating"]
numeric_features = ["kills", "deaths", "assists", "kills_deaths", "average_damage_per_round", "first_kills", "first_deaths", "tier"]
categorical_features = ["agent", "side"]

def replace_percentage(df, column):
    df[column] = df[column].str.replace('%', '').astype(float)
    return df

replace_kill_assist_trade_survive_ratio_transformer = FunctionTransformer(replace_percentage, kw_args={'column': 'kill_assist_trade_survive_ratio'})
replace_headshot_ratio_transformer = FunctionTransformer(replace_percentage, kw_args={'column': 'headshot_ratio'})

numeric_transformer = Pipeline(
    steps=[
        ("Median Imputer", SimpleImputer(strategy="median")),
    ]
)

categorical_transformer = Pipeline(
    steps=[
        ("Modal Imputer", SimpleImputer(strategy="most_frequent")),
        ("One-Hot Encoder", OneHotEncoder(handle_unknown="ignore")),
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ('replace_kill_assist_trade_survive_ratio', replace_kill_assist_trade_survive_ratio_transformer, ['kill_assist_trade_survive_ratio']),
        ('replace_headshot_ratio', replace_headshot_ratio_transformer, ['headshot_ratio']),
        ('column_dropper', 'drop', columns_to_drop),
        ("Numeric Transformer", numeric_transformer, numeric_features),
        ("Categorical Transformer", categorical_transformer, categorical_features),
    ],
    remainder="passthrough",
)

In [33]:
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=5000))
])

In [34]:
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('replace_kill_assist_trade_survive_ratio',
                                                  FunctionTransformer(func=<function replace_percentage at 0x17e0a5300>,
                                                                      kw_args={'column': 'kill_assist_trade_survive_ratio'}),
                                                  ['kill_assist_trade_survive_ratio']),
                                                 ('replace_headshot_ratio',
                                                  FunctionTransformer(func=<function r...
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['kills', 'deaths', 'assists',
                                                   'kills_deaths',
                                                   'average_damage_per_round',
                                                   'first_kills',
                                                   'first_deaths', 'tier']),
                                                 ('Categorical Transformer',
                                                  Pipeline(steps=[('Modal '
                                                                   'Imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('One-Hot '
                                                                   'Encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['agent', 'side'])])),
                ('classifier', LogisticRegression(max_iter=5000))])

In [35]:
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)

In [36]:
accuracy= accuracy_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_prob[:,1])
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("MSE:", mse)
print("Classification Report:\n", report)

Accuracy: 0.7093284603678919
MSE: 0.19126639690134822
Classification Report:
               precision    recall  f1-score   support

       False       0.71      0.71      0.71     78150
        True       0.71      0.71      0.71     78907

    accuracy                           0.71    157057
   macro avg       0.71      0.71      0.71    157057
weighted avg       0.71      0.71      0.71    157057



In [37]:
from pickle import dump

with open("model.pkl", "wb") as f:
    dump(model, f, protocol=5)